In [ ]:
!pip install polyglot
!pip install pycld2
!apt-get install python-numpy libicu-dev
!pip install -U git+https://github.com/aboSamoor/polyglot.git@master

In [ ]:
from polyglot.detect import Detector
from polyglot.detect.base import logger as polyglot_logger
polyglot_logger.setLevel("ERROR") #makes polyglot go shh... about the 'Detector is not able to detect the language reliably.'

In [1]:
import pandas as pd
import numpy as np

# Let's go, girls!

In [ ]:
#just a tiny playground to test things out
a = 'Foto. Papa Kreutzwaldi õueteatri etendus TONTLA (tegelaste ühispilt). Võru, 2000.'
print(a)
print(Detector(a, quiet = True).language.code)
print(Detector(a, quiet = True).language.confidence)
for lang in Detector(a, quiet = True).languages:
  print(lang.confidence)

In [ ]:
#another tiny playground to learn pandas ^.^
df1 = pd.DataFrame()
i = 0
bads = [3, 7]
for i in range(10):
  if i not in bads:
    print(i)
    rida = [[i, i+5, 'kasukas']] 
    df1 = df1.append(rida)
print(df1)

#And then, suddenly - Data

In [ ]:
#texts = pd.read_csv('/content/drive/MyDrive/Machine Learning/ajapaik.tsv', escapechar="\\", sep='\t', encoding = 'UTF-8') #creates a utf-8 error when predicting :(
#texts2 = pd.read_csv('/content/drive/MyDrive/Machine Learning/ajapaik.txt', escapechar="\\", sep='\t') #exported from excel. the computer says no
texts1 = pd.read_csv('/content/drive/MyDrive/Machine Learning/ajapaik1.txt', escapechar="\\", sep='\t', encoding = 'UTF-8') #removed a few odd characters, retyped one ž etc, copied to a new file
texts1.head() #is it nice? 
#yes it is 

,description,title,id
0,NaN,Vaade Pika Hermanni tornile edelast.,486766.0
1,NaN,"Raekoja plats, vaade Raekojale Voorimehe tänav...",486765.0
2,NaN,"Pirita kloostri varemed, esiplaanil jõgi ja pr...",486764.0
3,NaN,"Kadrioru loss, vaade pargipoolsete väravate ju...",486763.0
4,NaN,"Vaade Toompea nõlvale, vasakul Oleviste.",486762.0


#Polyglot whoop-whoop!

In [ ]:
col = ['id','description','title','lang_polyglot','conf_polyglot'] #this is the main output dataframe, later we can add more columns
df = pd.DataFrame(columns = col)

for index, line in texts1.iterrows():
  
  lang = Detector(str(line['description']) + str(line['title']), quiet = True).language.code
  conf = Detector(str(line['description']) + str(line['title']), quiet = True).language.confidence                
    
  newrow = [[line['id'], line['description'], line['title'], lang, conf]]
  #print(newrow) #where it gets stuck
  df = df.append(pd.DataFrame({'id': line['id'], 'description': line['description'], 'title': line['title'], 'lang_polyglot': lang, 'conf_polyglot': conf}, index = [0]), ignore_index = True)

KeyboardInterrupt: ignored

In [ ]:
df.tail() #look at the tail to know where it stopped
#stopped it by hand after 4 hours of runtime
#if good create an output file: 
df.to_csv(r'polyglot_out.csv', index = False) 

#FastText whoop whoop!

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git 
!cd fastText
!pip install fastText
!wget -O lid.176.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin


fatal: destination path 'fastText' already exists and is not an empty directory.
--2021-12-12 10:20:50--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘lid.176.bin’

lid.176.bin         100%[===================>] 125.18M  22.1MB/s    in 6.4s    

2021-12-12 10:20:57 (19.6 MB/s) - ‘lid.176.bin’ saved [131266198/131266198]



In [ ]:
#a tiny playground to test things out
import fasttext

lid = fasttext.load_model("lid.176.bin")

b = lid.predict(a)[0][0].split('_')[-1]
print(b)

et


In [ ]:
import fasttext
fast_out = pd.DataFrame(columns = ['id','lang_fast','conf_fast'])

lid = fasttext.load_model("lid.176.bin")

for index, line in texts.iterrows():
  #lang = lid.predict(str(line['description']).replace("\n"," ") + str(line['title']).replace("\n"," "))[0][0].split('_')[-1]
  #conf = lid.predict(str(line['description']).replace("\n"," ") + str(line['title']).replace("\n"," "))[1][0]
  if line['title'] == "": #if no title, use description
    lang = lid.predict(str(line['description']).replace("\n"," "))[0][0].split('_')[-1]
    conf = lid.predict(str(line['description']).replace("\n"," "))[1][0]
  else:
    lang = lid.predict(str(line['title']).replace("\n"," "))[0][0].split('_')[-1]
    conf = lid.predict(str(line['title']).replace("\n"," "))[1][0]
    
  newrow = [[line['id'], lang, conf]]
  #print(newrow) #where it gets stuck
  fast_out = fast_out.append(pd.DataFrame({'id': line['id'], 'lang_fast': lang, 'conf_fast': conf}, index = [0]), ignore_index = True)

In [ ]:
fast_out.tail()
fast_out.to_csv(r'fast_out.csv', index = False)

,id,lang_fast,conf_fast
453959,1700,ht,0.987593
453960,1699,et,0.889045
453961,1698,en,0.373775
453962,1697,et,0.286782
453963,1696 } ],et,0.283266


#Fasttext second model

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git 
!cd fastText
!pip install fastText
!wget -O lid.176.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz

In [ ]:
import fasttext
fast_out = pd.DataFrame(columns = ['id','lang_fast','conf_fast'])

lid = fasttext.load_model("lid.176.bin")

for index, line in texts.iterrows():
  #lang = lid.predict(str(line['description']).replace("\n"," ") + str(line['title']).replace("\n"," "))[0][0].split('_')[-1]
  #conf = lid.predict(str(line['description']).replace("\n"," ") + str(line['title']).replace("\n"," "))[1][0]
  if line['title'] == "": #if no title, use description
    lang = lid.predict(str(line['description']).replace("\n"," "))[0][0].split('_')[-1]
    conf = lid.predict(str(line['description']).replace("\n"," "))[1][0]
  else:
    lang = lid.predict(str(line['title']).replace("\n"," "))[0][0].split('_')[-1]
    conf = lid.predict(str(line['title']).replace("\n"," "))[1][0]
    
  newrow = [[line['id'], lang, conf]]
  #print(newrow) #where it gets stuck
  fast_out = fast_out.append(pd.DataFrame({'id': line['id'], 'lang_fast': lang, 'conf_fast': conf}, index = [0]), ignore_index = True)

# Lets combine some stuffs

In [2]:
import pandas as pd
import numpy as np

In [3]:

predictions1 = pd.read_csv('/content/drive/MyDrive/Machine Learning/polyglot_out.csv', escapechar="\\", sep=',', encoding = 'UTF-8') 
predictions1['conf_polyglot2'] = predictions1['conf_polyglot']/100
predictions2 = pd.read_csv('/content/drive/MyDrive/Machine Learning/fast_out.csv', escapechar="\\", sep=',', encoding = 'UTF-8') 
predictions3 = pd.read_csv('/content/drive/MyDrive/Machine Learning/fast_out2.csv', escapechar="\\", sep=',', encoding = 'UTF-8') 

predictions3 = predictions3.rename(columns={"lang_fast": "lang_fast_2", "conf_fast": "conf_fast2"}, errors = 'raise')

predictions0 = pd.merge(predictions1, predictions3, on = 'id')
predictions = pd.merge(predictions2, predictions3, on = 'id')

In [4]:
#write to file and do the last merge offline
predictions.to_csv(r'predictions.csv', index = False)
predictions0.to_csv(r'predictions0.csv', index = False)

In [ ]:
#don't run this or colab will crash
predictions = pd.merge(predictions0, predictions, on = 'id')